# Capítulo 8 - Detecção de anomalias

## Bibliotecas básicas e outras inicializações

In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')

## Carregamento dos dados

In [2]:
dados = pd.read_csv('./datasets/HealthcareProviders.csv')
print(f'Linhas: {dados.shape[0]} | Colunas: {dados.shape[1]}')
dados.sample()

Linhas: 100000 | Colunas: 27


,index,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Middle Initial of the Provider,Credentials of the Provider,Gender of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,Street Address 2 of the Provider,...,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
38246,5261126,1538166996,LEWIS,DOUGLAS,E,D.O.,M,I,6700 SW 9TH AVE,NaN,...,85025,"Complete blood cell count (red cells, white bl...",N,160,125,160,10.66,40,10.45,10.45


## Dicionário dos dados

- **index**: código do registro.

- **National Provider Identifier (NPI)**: identificador de Provedor Nacional para o provedor de desempenho no sinistro. O provedor NPI é o identificador numérico registrado no NPPES.

- **Last Name/Organization Name of the Provider**: quando o provedor está registrado no NPPES como um indivíduo (código de tipo de entidade = 'I') este é o sobrenome do provedor e quando o provedor está registrado como uma organização (código de tipo de entidade = 'O') este é o nome da organização.

- **First Name of the Provider**: quando o provedor está registrado no NPPES como um indivíduo (código de tipo de entidade = 'I') este é o primeiro nome do provedor e quando o provedor está registrado como uma organização (código de tipo de entidade = 'O'), estará em branco.

- **Middle Initial of the Provider**: inicial do meio do fornecedor.
 
- **Credentials of the Provider**: quando o provedor está registrado no NPPES como um indivíduo (código de tipo de entidade = 'I') essas são as credenciais do provedor e quando o provedor está registrado como uma organização (código de tipo de entidade = 'O'), estará em branco.

- **Gender of the Provider**: quando o provedor está registrado no NPPES como um indivíduo (código de tipo de entidade = 'I') este é o gênero do provedor e quando o provedor está registrado como uma organização (código de tipo de entidade = 'O'), estará em branco.

- **Entity Type of the Provider**: tipo de entidade relatada no NPPES. Código de entidade 'I' identifica provedores registrados como indivíduos e o código de entidade 'O' identifica provedores registrados como organizações.

- **Street Address 1 of the Provider**: a primeira linha do endereço do provedor, conforme relatado no NPPES.

- **Street Address 2 of the Provider**: a segunda linha do endereço do provedor, conforme relatado no NPPES.

- **City of the Provider**: cidade onde o provedor está localizado, conforme informado no NPPES.

- **Zip Code of the Provider**: código postal do provedor, conforme relatado no NPPES.

- **State Code of the Provider***: estado onde o provedor está localizado, conforme relatado no NPPES. Os 50 estados dos EUA e o Distrito de Columbia são informados pela abreviatura postal estadual. Os seguintes valores são usados para todas as outras áreas: 'XX' = 'Desconhecido', 'AA' = 'Forças Armadas Central / América do Sul', 'AE' = 'Forças Armadas da Europa', 'AP' = 'Forças Armadas do Pacífico', 'AS' = 'Samoa Americana', 'GU' = 'Guam', 'MP' = 'Ilhas Marianas do Norte', 'PR' = 'Porto Rico', 'VI' = 'Ilhas Virgens', 'ZZ' = 'País Estrangeiro'.

- **Country Code of the Provider**: país onde o provedor está localizado, conforme relatado no NPPES. O código do país será 'EUA' para qualquer estado ou posse dos EUA. Para países estrangeiros os valores do país do provedor incluem o seguinte: AE = Emirados Árabes Unidos, IT = Itália, AG = Antigua, JO = Jordan, AR = Argentina, JP = Japão, AU = Austrália, KR = Coréia, BO = Bolívia, KW = Kuwait, BR = Brasil, KY = Ilhas Cayman, CA = Canadá, LB = Líbano, CH = Suíça, MX = México, CN = China, NL = Holanda, CO = Colômbia, NO = Noruega, DE = Alemanha, NZ = Nova Zelândia, ES = Espanha, PA = Panamá, FR = França, PK = Paquistão, GB = Grã-Bretanha, RW = Ruanda, GR = Grécia, SA = Arábia Saudita, HU = Hungria, SY = Síria, IL = Israel, TH = Tailândia, IN = Índia, TR = Turquia, IS = Islândia, VE = Venezuela.

- **Provider Type**: derivado do código de especialidade do provedor relatado na reclamação.

- **Medicare Participation Indicator**: identifica se o provedor participa do Medicare e/ou aceita a atribuição de valores permitidos do Medicare.

- **Place of Service***: identifica se o local de serviço apresentado nas reivindicações é uma instalação ('F') ou não-instalação ('O'). A ausência de instalações é geralmente um ambiente de escritório, no entanto, outras entidades estão incluídas em non-facilit.

- **HCPCS Code**: código HCPCS usado para identificar o serviço médico específico fornecido pelo provedor.

- **HCPCS Description**: descrição do código HCPCS para o serviço médico específico fornecido pelo provedor.

- **HCPCS Drug Indicator**: identifica se o código HCPCS para o serviço específico fornecido pelo provedor é um HCPCS listado no arquivo de preço médio de venda de medicamentos (ASP) do Medicare Parte B.

- **Number of Services**: número de serviços prestados. Observe que as métricas usadas para contar o número fornecido podem variar de serviço para serviço.

- **Number of Medicare Beneficiaries**: número de beneficiários distintos do Medicare que recebem o serviço.

- **Number of Distinct Medicare Beneficiary/Per Day Services**: número de beneficiários distintos do Medicare/serviços diários.

- **Average Medicare Allowed Amount**: média do valor permitido pelo Medicare para o serviço.

- **Average Submitted Charge Amount**: média das cobranças que o provedor enviou para o serviço.

- **Average Medicare Payment Amount'**: valor médio que o Medicare pagou depois que os valores de franquia e cosseguro foram deduzidos para o serviço de item de linha.

- **Average Medicare Standardized Amount**: quantidade média padronizada do Medicare.

### Informação dos dados

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   index                                                     100000 non-null  int64  
 1   National Provider Identifier                              100000 non-null  int64  
 2   Last Name/Organization Name of the Provider               100000 non-null  object 
 3   First Name of the Provider                                95745 non-null   object 
 4   Middle Initial of the Provider                            70669 non-null   object 
 5   Credentials of the Provider                               92791 non-null   object 
 6   Gender of the Provider                                    95746 non-null   object 
 7   Entity Type of the Provider                               100000 non-null  object 
 8   Stree

### Seleção de recursos

In [4]:
dropCols = ['index', 'National Provider Identifier', 'Last Name/Organization Name of the Provider',
            'First Name of the Provider', 'Middle Initial of the Provider', 'Street Address 1 of the Provider',
            'Street Address 2 of the Provider', 'Zip Code of the Provider', 'HCPCS Code']

dados.drop(dropCols, inplace=True, axis=1)
dados.reset_index(drop=True, inplace=True)

### Colunas numéricas

In [5]:
cols = [
     'Number of Services', 
     'Number of Medicare Beneficiaries', 
     'Number of Distinct Medicare Beneficiary/Per Day Services', 
     'Average Medicare Allowed Amount', 
     'Average Submitted Charge Amount',
     'Average Medicare Payment Amount',
     'Average Medicare Standardized Amount'
]

def removeComma(x):
    return x.replace(',', '')

for c in cols:
    dados[c] = pd.to_numeric(dados[c].apply(lambda x: removeComma(x)), errors= 'ignore')

### Remoção dos nulos

In [6]:
cols = ['Credentials of the Provider', 'Gender of the Provider']
for c in cols:
    dados[c] = dados[c].fillna(dados[c].mode()[0])
    
print(dados.isna().sum())

Credentials of the Provider                                 0
Gender of the Provider                                      0
Entity Type of the Provider                                 0
City of the Provider                                        0
State Code of the Provider                                  0
Country Code of the Provider                                0
Provider Type                                               0
Medicare Participation Indicator                            0
Place of Service                                            0
HCPCS Description                                           0
HCPCS Drug Indicator                                        0
Number of Services                                          0
Number of Medicare Beneficiaries                            0
Number of Distinct Medicare Beneficiary/Per Day Services    0
Average Medicare Allowed Amount                             0
Average Submitted Charge Amount                             0
Average 

### Colunas categóricas

In [7]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

def transformCategoricalIntoNumeric(data):
    cols = [var for var in data.columns if data[var].dtype == 'O']
    
    for col in cols:
        encoder = ce.BinaryEncoder(cols=[col])
        dfbin = encoder.fit_transform(data[col])
        data = pd.concat([data, dfbin], axis=1)
        del data[col]
     
    data_columns = data.columns
    std = StandardScaler()
    data = std.fit_transform(data)
    data = pd.DataFrame(data, columns=data_columns)
    return data

dados = transformCategoricalIntoNumeric(dados)

### Visão dos dados

In [8]:
print(f'Linhas: {dados.shape[0]} | Colunas: {dados.shape[1]}')
dados.head()

Linhas: 100000 | Colunas: 74


,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,Credentials of the Provider_0,Credentials of the Provider_1,Credentials of the Provider_2,...,HCPCS Description_5,HCPCS Description_6,HCPCS Description_7,HCPCS Description_8,HCPCS Description_9,HCPCS Description_10,HCPCS Description_11,HCPCS Description_12,HCPCS Drug Indicator_0,HCPCS Drug Indicator_1
0,-0.085301,-0.059308,-0.070183,0.385450,-0.046433,0.400082,0.414299,0.0,-0.097882,-0.115257,...,-0.674247,-0.734606,-0.985032,-1.05161,-0.963872,-0.893767,-0.984165,0.987083,-0.257051,0.257051
1,-0.025939,0.076775,0.020049,0.086673,0.182805,0.207649,0.286359,0.0,-0.097882,-0.115257,...,-0.674247,-0.734606,-0.985032,-1.05161,-0.963872,-0.893767,1.016089,-1.013086,-0.257051,0.257051
2,-0.083296,-0.069222,-0.067135,-0.041922,-0.187794,-0.064687,-0.087154,0.0,-0.097882,-0.115257,...,-0.674247,-0.734606,-0.985032,-1.05161,-0.963872,-0.893767,1.016089,0.987083,-0.257051,0.257051
3,-0.088109,-0.064716,-0.074451,-0.380709,-0.328957,-0.370166,-0.372921,0.0,-0.097882,-0.115257,...,-0.674247,-0.734606,-0.985032,-1.05161,-0.963872,1.118859,-0.984165,-1.013086,-0.257051,0.257051
4,-0.082895,-0.059308,-0.067744,-0.291221,-0.296019,-0.289505,-0.294800,0.0,-0.097882,-0.115257,...,-0.674247,-0.734606,-0.985032,-1.05161,-0.963872,1.118859,-0.984165,0.987083,-0.257051,0.257051
